Comme nous venons de le voir, Kedro est un outil puissant. Le premier pipeline que nous avons construit permettait d'encoder les données pour qu'elles soit utilisées par un modèle. Pour continuer à construire le pipeline ML, nous allons définir ici le **pipeline d'entraînement de modèles**, qui pourra être combiné avec celui que nous avons déjà fait sur l'encodage.

<blockquote><p>🙋 <b>Ce que nous allons faire</b></p>
<ul>
    <li>Développer le pipeline d'entraînement de modèles</li>
    <li>Construire le pipeline ML de la collecte des données jusqu'à la modélisation</li>
    <li>Appliquer les bonnes pratiques de développement</li>
</ul>
</blockquote>

<img src="https://media.giphy.com/media/l46CwEYnbFtFfjZNS/giphy.gif" />

## Entraînement du modèle

Nous avions appliqué de l'AutoML sur un LightGBM avec des Jupyter Notebooks. Notre objectif est maintenant de le faire directement dans le projet Kedro. Nous allons construire un pipeline nommé `training` qui est chargé de de déterminer un modèle optimal.

<div class="alert alert-block alert-info">
    Dans la construction, le pipeline <code>training</code> intervient après le pipeline <code>processing</code>, mais l'intérêt de séparer le pipeline ML en deux est de pouvoir exécuter un seul des deux pipelines au besoin.
</div>

Créons un dossier `training` dans `src/purchase_predict/pipelines`. Notre premier fichier `nodes.py` contient toutes les fonctions nécessaires pour entraîner le module. Pour améliorer la productivité et faciliter la maintenance du code, nous allons y créer trois fonctions.

- La première fonction `train_model` va entraîner une instance de modèle selon des hyper-paramètres spécifiques sur un ensemble d'entraînement.
- La seconde fonction `optimize_hyp` va déterminer les hyper-paramètres qui maximisent le score d'un modèle à partir d'une métrique spécifique.
- La dernière fonction `auto_ml` sera ensuite utilisée par le node Kedro pour entraîner un modèle optimisé.

Commençons par importer les librairies nécessaires aux fonctions que nous développerons.

Créons une variable `MODELS` dans laquelle nous allons spécifier une liste de modèles candidats. Pour commencer, nous n'utiliserons qu'un seul modèle LightGBM, mais nous pourrions également calibrer d'autres modèles comme XGBoost, CatBoost, Random Forest. Nous définissons un dictionnaire pour chaque modèle.

- `name` est le nom du modèle.
- `class` est l'objet Python permettant d'instancier le modèle.
- `params` représente l'espace de recherche pour l'optimisation des hyper-paramètres.
- `override_schemas` est un champ qui contient les hyper-paramètres dont le type doit être modifié avant d'entraîner le modèle.

Par exemple, l'hyper-paramètre `max_depth` doit être entier, mais un point de l'espace peut produire un flottant ($10.0$ au lieu de $10$). Dans ce cas, il faut forcer la conversion en entier pour éviter une erreur générée par LightGBM.

Bien que le pipeline d'entraînement du modèle ne possédera qu'un node, nous pouvons tout à fait créer plusieurs fonctions dans le fichier `nodes.py` qui vont être appelées par la fonction du node.

Commençons par la première fonction `train_model`.

Comme nous l'avons évoqué plus haut, nous forçons la conversion de certains hyper-paramètres. On récupère les noms des hyper-paramètres dont le type est surchargé, puis ils sont convertis vers le type cible (ici, uniquement des entiers). Le reste de la fonction est explicitive, car il s'agit juste d'instancier le modèle et d'appeler la fonction `fit`.

Voyons maintenant la deuxième fonction `optimize_hyp`.

Tout d'abord, nous récupérons la base d'apprentissage $(X, y)$ à partir de l'argument `dataset`. Nous définissons ensuite la fonction `objective` dont on cherche une minimisation. À chaque évaluation, elle réalise un $k$-Fold sur le modèle candidat, puis stocke le score, calculée à partir de la métrique, dans la liste `scores_test`.

Cette liste contient tous les scores sur les ensembles de test du $k$-Fold. Ici, nous aurons uniquement $4$ scores à chaque exécution de la fonction. Nous retournons donc le score moyen des modèles sur les ensembles de test.

Pour terminer, nous retournons le résultat de `fmin`, c'est-à-dire le jeu d'hyper-paramètres qui maximise la fonction `objective`.

Enfin, la dernière fonction `auto_ml`, qui sera utilisée par le node Kedro.

Il s'agit simplement d'une exéuction séquencée en plusieurs étapes. Pour chaque modèle candidat que l'on souhaite optimiser, il y a trois étapes.

- On détermine les hyper-paramètres optimaux avec `optimize_hyp` sur la base d'apprentissage $(X, y)$.
- Une fois les hyper-paramètres optimaux trouvés, une instance du modèle est calibré avec ces derniers sur le sous-échantillon d'entraînement.
- Le score du modèle est calculé sur le sous-échantillon de test et stocké dans une liste.

À la fin, la liste `opt_models` contiendra tous les modèles optimisé de chaque modèle candidat. Si l'on souhaite tester un LightGBM, un XGBoost et un CatBoost, alors `opt_models` contiendra trois éléments, chacun étant le modèle optimisé de la classe d'algorithme.

Pour terminer, on tourne le meilleur modèle parmi tous les modèles optimisés.

### Création du pipeline

Dernière étape : construire le pipeline dans le fichier `pipeline.py`. Avant toute chose, nous allons définir le modèle (optimisé) dans le Data Catalog afin de l'enregistrer une fois entraîné. On utilise le type `pickle.PickleDataSet` pour enregistrer un modèle au format binaire sur le disque.

De même, nous allons définir un paramère Kedro pour le nombre d'itérations lors de la phase d'AutoML.

Pour le pipeline, il n'y a qu'un seul node à définir.

On installera les packages nécessaires avec `pip install lightgbm hyperopt` en vérifiant que l'on soit bien dans l'environnement virtuel.

<div class="alert alert-info">
    Pour mettre à jour la visualisation des pipelines, il faut relancer le processus <code>kedro viz</code> dans un terminal (on le stoppera au préalable avec <code>Ctrl+C</code>).
</div>

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/kedro3.png" />

Comme nous l'avions fait pour le pipeline `processing`, nous devons définir le pipeline dans le fichier `hooks.py`.

Après l'importation du pipeline, mettons à jour le corps de la fonction `register_pipelines`.

Il ne reste plus qu'à exécuter le pipeline. Nous allons au total entraîner 40 modèles puisque nous avons un $4$-Fold et 10 itérations : le temps d'exécution sera d'environ 3 à 5 minutes.

## Pipeline de chargement

Dans le pipeline `processing`, on suppose que le jeu de données `primary.csv` existe déjà. Or, il s'agit pour l'instant du jeu de données **de l'échantillon**, et non de celui calculé sur un historique de 7 jours. Nous devons au préalable créer un troisième pipeline qui interviendra en tout premier.

Créons un dossier `loading` avec les fichiers `nodes.py` et `pipeline.py` pour le pipeline de même nom.

Là-aussi, il nous faut installer `pip install google-cloud-storage` dans l'environnement virtuel.

Avec `storage_client`, nous téléchargeons en local dans le dossier `/tmp` (dossier de fichiers temporaires) tous les fichiers CSV dans le bucket spécifié. À l'aide de `glob`, nous parcourons tous ces fichiers téléchargés et les concaténons dans un seul et unique DataFrame `df`.

Puisqu'il n'y a qu'un seul noeud, le pipeline est rapide à définir.

Nous allons rajouter deux paramètres dans le fichier `parameters.yml`.

- Le paramètre `gcp_project_id` contenant le nom du projet Google Cloud.
- Le paramètre `gcs_primary_folder` qui donne **le dossier** contenant les fichiers CSV transformés, prêts à être encodés.

Comme toujours, pour rajouter le pipeline, il faut configurer le fichier `hooks.py`.

Nous avons donc un total de trois pipelines.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/pipeline_ml1.png" />

Essayons d'exécuter le pipeline `loading`.

La description de l'erreur est explicite : `Anonymous caller does not have storage.objects.list access` car nous **ne sommes pas authentifié**. Afin d'authentifier une application pour accéder à une ou plusieurs ressources Google Cloud, nous devons créer un <a href="https://console.cloud.google.com/identity/serviceaccounts?project=training-ml-engineer" target="_blank">compte de service</a>.

### Création d'un compte de service

Une règle de sécurité en Cloud Computing consiste à créer des **rôles** en fonction des services et des cas d'utilisation. Sous GCP, les **comptes de services** permettent de définir des rôles aux applications qui vont faire appel à certains services, avec un certain niveau d'habilitation tout en garantissant une sécurité élevée.

Nous allons nommer ce compte de service `purchase-predict` avec le rôle *Lecteur des objets de l'espace de stockage*, ce qui autorisera la lecture des données depuis l'extérieur.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/pipeline_ml2.png" />

La description du compte de service n'est pas obligatoire, mais elle permet de présenter rapidement quelles applications vont utiliser ce compte de service car cela n'est pas toujours clair lorsqu'il y en a beaucoup.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/pipeline_ml3.png" />

En sécurité du Cloud, il faut adopter le **principe du moindre privilège** : il faut donner uniquement les accès nécessaires, c'est-à-dire juste ce qu'il faut pour que l'application fonctionne, jamais plus.

Après avoir crée le compte de service, nous pouvons créer une clé au format JSON et la télécharger.

<div class="alert alert-block alert-danger">
    La clé du compte de service est équivalent à un mot de passe. Il ne faut jamais la divulguer ou la laisser dans un projet où Git ajouterai ce fichier à un dépôt distant.
</div>

Heureusement pour nous, Kedro a prévu de cas de figure. Tous les mots de passes et clés de services, lorsqu'ils sont utilisé sur des environnement de développement, doivent être placés dans le dossier `conf/local`, qui est ignoré par Git. Ajoutons le fichier `service-account.json` dans ce dossier en insérant le contenu de la clé JSON téléchargée.

Il reste maintenant à spécifier le chemin d'accès à la clé du compte de service dans la variable d'environnement `GOOGLE_APPLICATION_CREDENTIALS`. À noter que cette manipulation sera nécessaire **lorsque l'on démarre un nouveau terminal**.

Essayons d'exécuter à nouveau le pipeline.

Tout a bien fonctionné : le fichier `primary.csv` correspond bien au jeu de données construit avec PySpark.

Au total, nous comptabilisons trois pipelines.

- Le pipeline `loading` qui charger les fichiers CSV depuis le bucket Cloud Storage pour créer le fichier `primary.csv`.
- Le pipeline `processing` qui va encoder le jeu de données `primary.csv`.
- Le pipeline `training` qui va entraîner un modèle.

L'avantage de la représentation de ces trois pipelines est **la flexibilité**. Si je souhaite ajouter un XGBoost par exemple, je n'aurai besoin que de relance le pipeline `training`. Si je change de méthodes d'encodages, je n'aurai pas besoin de relancer le pipeline `loading`. L'autre aspect important est **l'homogénéité des traitements** : en regroupant toutes les opérations dans trois pipelines, nous nous assurons que les données au tout début de la chaîne vont subir exactement le même traitement, quelle que soit la date d'exécution ou les données en entrée.

Nous pouvons alors créer un pipeline `global`, qui est la fusion des trois pipelines séquencés.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/pipeline_ml4.png" />

Nous pouvons ensuite lancer le pipeline `global`. À noter que le pipeline `training` prendra plus de temps puisque le jeu de données est plus conséquent.

## Linting et Refactoring

À partir de maintenant, les codes qui vont être produits seront utilisés dans des environnements de production. Il est **nécessaire** que le code respecte les normes de PEP 8, telles que nous les avions vues avec `flake8` et `black`. Installons ces deux packages.

Nous allons ensuite créer deux fichiers.

- Le fichier `.flake8` à la racine du projet configure les paramètres propres à `flake8`. En particulier, nous pouvons par exemple ignorer certaines erreurs de PEP 8 ou, à l'inverse, ajouter ses propres spécifications.
- Le fichier `.toml`, lui aussi à la racine du projet, configure les paramètres pour `black`. À noter que dans les deux fichiers, il est important de préciser la même taille pour les lignes. Là où `flake8` générera une erreur, `black` découpera la ligne actuelle en plusieurs.

Commençons par le fichier `.flake8`.

Nous choisissons volontairement un nombre de caractères maximal à 120. La complexité, qui est un calcul réalisé en fonction du nombre d'identations maximales et de variables utilisées, est fixée à 16. Enfin, nous définissons les dossiers et fichiers à exclure de l'analyse.

Le contenu du fichier `.toml` est très similaire.

La seule différence est que l'on choisit également d'y inclure certains fichiers dont l'extension est `.pyi`. Exécutons tout d'abord `flake8`.

Nous remarquons que la majorité des erreurs et avertissements peuvent être corrigés par un formatage de code avec `black`.

En exécutant à nouveau `flake8`, il ne devrait y avoir aucune sortie console, signifiant que tous les codes sources respectent PEP 8.

> ❓ Est-ce que l'on doit tout le temps exécuter ces deux commandes ?

Les bonnes pratiques en développement logiciel, c'est de toujours publier un code qui respecte au maximum les normes, notamment PEP 8 dans le cas de Python. Le plus adéquat ici serait d'exécuter automatiquement `black` puis `flake8` à chaque commit de Git. Il y aurait alors deux possibilités.

- Le code ne respecte pas la norme PEP 8, et le commit n'est pas accepté.
- Le code respecte la norme PEP 8, et le commit est accepté **en local**.

Il faudra ensuite pousser le code local vers le dépôt distant. Et tout ceci peut être réalisé avec `pre-commit`.

Il s'agit d'un utilitaire qui permet d'exécuter des traitements avant, pendant et après des actions Git (lors d'un commit, d'un push, etc). Pour cela, nous configurons les événements à surveiller en spécifiant, dans le fichier `.pre-commit-config.yaml`, toujours à la racine du projet.



Pour initialiser l'environnement et installer les déclencheurs, il suffit d'exécuter `pre-commit install` dans la console. Cette opération nécessite quelques dizaines de secondes. Dorénavant, à chaque commit, `black` puis `flake8` sont exécutés et cela garantit que le code qui sera par la suite poussé vers le dépôt distant respecte la norme PEP 8.

Dorénavant, à la moindre modification de fichier, lors d'un commit, tous les codes Python seront inspectés.

Pour terminer, nous pouvons pousser le code vers le dépôt distant.

À noter qu'à chaque nouveau terminal, il faut relancer l'agent SSH et lui spécifier la clée privée pour pouvoir avoir les droits sur le dépôt distant.

## ✔️ Conclusion

Notre pipeline ML est enfin réalisé avec Kedro.

- Tout d'abord, nous avons construit le pipeline qui entraîne des modèles.
- Ensuite, nous avons combiné tous les pipelines (collecte, encodage et entraînement) en un seul.
- Pour finir, nous avons appliqué les bonnes pratiques de développement en vérifiant les codes Python avec la norme PEP 8.

> ➡️ Au programme des prochaines activités : les **dépôts de modèles** ! Mais avant, il est nécessaire d'aborder un point pas toujours plaisant, mais pour autant très important : les **tests logiciels**.